# Intermediate Neural Network in TensorFlow

In this notebook, we convert our [intermediate-depth neural network](https://github.com/the-deep-learners/TensorFlow-LiveLessons/blob/master/notebooks/intermediate_net_in_keras.ipynb) from Keras to TensorFlow following Aymeric Damien's [MLP Notebook](https://github.com/aymericdamien/TensorFlow-Examples/blob/master/notebooks/3_NeuralNetworks/multilayer_perceptron.ipynb) style.

#### Load dependencies

In [1]:
import numpy as np
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

#### Load data

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


#### Set neural network hyperparameters (tidier at top of file!)

In [3]:
lr = 0.1
epochs = 20
batch_size = 128
weight_initializer = tf.contrib.layers.xavier_initializer()

#### Set number of neurons for each layer

In [4]:
n_input = 784
n_hidden_1 = 64
n_hidden_2 = 64
n_classes = 10

#### Define placeholders for inputs and labels

In [5]:
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

#### Design neural network architecture

In [6]:
def network(x, weights, biases):
    
    # hidden layer 1: dense with ReLU activation
    dense_1 = tf.add(tf.matmul(x, weights['W1']), biases['b1'])
    dense_1 = tf.nn.relu(dense_1)
    
    # hidden layer 2: dense with ReLU activation
    dense_2 = tf.add(tf.matmul(dense_1, weights['W2']), biases['b2'])
    dense_2 = tf.nn.relu(dense_2)
    
    # linear output layer (softmax )
    out_layer = tf.add(tf.matmul(dense_2, weights['W_out']), biases['b_out'])
    
    return out_layer

#### Define dictionaries for storing weights and biases for each layer, initialized with random values

In [7]:
weight_dict = {
    'W1': tf.get_variable('W1', [n_input, n_hidden_1], initializer=weight_initializer),
    'W2': tf.get_variable('W2', [n_hidden_1, n_hidden_2], initializer=weight_initializer),
    'W_out': tf.get_variable('W_out', [n_hidden_2, n_classes], initializer=weight_initializer)
}
bias_dict = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])), 
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'b_out': tf.Variable(tf.random_normal([n_classes]))
}

#### Build model

In [8]:
predictions = network(x, weights=weight_dict, biases=bias_dict)

#### Define model loss and optimizer

In [9]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr).minimize(cost)

#### Initialize variables

In [10]:
initializer_op = tf.global_variables_initializer()

#### Train the network in a session

In [11]:
with tf.Session() as session:
    session.run(initializer_op)
    
    for epoch in range(epochs):
        
        avg_cost = 0.0 # track cost to monitor performance during training
        
        # loop over all batches of the epoch:
        n_batches = int(mnist.train.num_examples/batch_size)
        for i in range(n_batches):
            
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            
            # feed batch data to run optimization and fetch cost: 
            _, batch_cost = session.run([optimizer, cost], feed_dict={x: batch_x, y: batch_y})
            
            # compute average loss:
            avg_cost += batch_cost / n_batches
            
        # output logs at end of each epoch of training:
        print("Epoch:", '%03d' % (epoch+1), "/", epochs, "cost=", "{:.3f}".format(avg_cost))
    
    print("Training Complete. Testing Model.")
    
    # calculate accuracy by identifying test cases where the model's highest-probability class matches the true y label: 
    correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))

Epoch: 001 / 20 cost= 0.521
Epoch: 002 / 20 cost= 0.260
Epoch: 003 / 20 cost= 0.203
Epoch: 004 / 20 cost= 0.166
Epoch: 005 / 20 cost= 0.141
Epoch: 006 / 20 cost= 0.123
Epoch: 007 / 20 cost= 0.109
Epoch: 008 / 20 cost= 0.098
Epoch: 009 / 20 cost= 0.089
Epoch: 010 / 20 cost= 0.081
Epoch: 011 / 20 cost= 0.074
Epoch: 012 / 20 cost= 0.068
Epoch: 013 / 20 cost= 0.063
Epoch: 014 / 20 cost= 0.059
Epoch: 015 / 20 cost= 0.055
Epoch: 016 / 20 cost= 0.050
Epoch: 017 / 20 cost= 0.047
Epoch: 018 / 20 cost= 0.044
Epoch: 019 / 20 cost= 0.040
Epoch: 020 / 20 cost= 0.038
Training Complete. Testing Model.
Accuracy: 0.9718
